In [2]:
import os
os.chdir('D://Projects-2026//Medical Chat-bot//MediCare-AI')
%pwd

'D:\\Projects-2026\\Medical Chat-bot\\MediCare-AI'

In [3]:
# Loading PDF files.
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Extract text from PDF file
def load_pdf_files(data):
    loader=DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [4]:
extracted_data = load_pdf_files("data")

In [5]:
# extracted_data

In [6]:
# len(extracted_data)

In [7]:
# Filter documents to minimal content and metadata.
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [8]:
minimal_docs= filter_to_minimal_docs(extracted_data)
minimal_docs

[Document(metadata={'source': 'data\\gynaec book.pdf'}, page_content=''),
 Document(metadata={'source': 'data\\gynaec book.pdf'}, page_content=''),
 Document(metadata={'source': 'data\\gynaec book.pdf'}, page_content='Copyright © 2013 by The McGraw-Hill Companies, Inc. All rights reserved. Except as permitted\nunder the United States Copyright Act of 1976, no part of this publication may be reproduced or\ndistributed in any form or by any means, or stored in a database or retrieval system, without the prior\nwritten permission of the publisher.\nISBN: 978-0-07-163969-9\nMHID:       0-07-163969-1\nThe material in this eBook also appears in the print version of this title: ISBN: 978-0-07-163856-2,\nMHID: 0-07-163856-3.\nAll trademarks are trademarks of their respective owners. Rather than put a trademark symbol after\nevery occurrence of a trademarked name, we use names in an editorial fashion only, and to the benefit\nof the trademark owner, with no intention of infringement of the trad

In [9]:
# Spitting the document in smaller chunks.
def text_split(minimal_docs):
    text_splitt = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=50,
    )
    text_chunk=text_splitt.split_documents(minimal_docs)
    return text_chunk

In [10]:
text_chunk = text_split(minimal_docs)

In [11]:
# Downloading and loading Embeddings model.
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    model_name="sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
    )
    return embeddings
embedding=download_embeddings()

C:\Users\hetde\AppData\Local\Temp\ipykernel_23320\2668003445.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [12]:
vector=embedding.embed_query("Hey")

In [13]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [14]:
# Accessing the .env values and making them available for use globally.
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [15]:
# Creating pinecone client.
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY
pc = Pinecone(api_key=pinecone_api_key)

In [16]:
# Creating Inedx in Pinecone
from pinecone import ServerlessSpec
index_name = "medicare-ai"
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
index = pc.Index(index_name)


In [ ]:
# # Creating vector embeddings in Pinecone Index.
# from langchain_pinecone import PineconeVectorStore

# docsearch = PineconeVectorStore.from_documents(
#     documents=text_chunk,
#     embedding=embedding,
#     index_name=index_name
# )

In [18]:
# Load Existing Index
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding=embedding
)

# Add more data to the existing Pinecone Index

In [19]:
dummyDoc = Document(
    page_content="For any other queries, that are not covered here, please consult you doctor.",
    metadata={"source": "Youtube"}
)
docsearch.add_documents([dummyDoc])

['ffd6d7f4-cd69-4f00-a265-6eee9034716e']

In [20]:
# Retriving based on similarity search.
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})
retriever_docs = retriever.invoke("What is Menopause?")
retriever_docs

[Document(id='c53fd1fe-534e-4297-abf4-2386bccb4ab1', metadata={'source': 'data\\gynaec book.pdf'}, page_content='59\nMenopause & Postmenopause\nLauren Nathan, MD\nESSENTIALS OF DIAGNOSIS\n Natural menopause diagnosed after 12 months of amenorrhea with no obvious pathologic cause\n Average age 51 years\n Estradiol <20 pg/mL and follicle-stimulation hormone level 21–100 mU/mL helpful in establishing\nthe diagnosis\n Induced menopause is defined as permanent cessation of menstruation after bilateral oophorectomy\nor ablation of ovarian function (ie, by chemotherapy or radiation)\n Premature menopause defined as menopause reached at or before age 40 and can be natural or\ninduced\n Perimenopause/menopause transition defined by menstrual cycle and hormonal changes that occur a\nfew years before and 12 months after the final menstrual period resulting from natural menopause\n May be associated with vasomotor symptoms, sleep disturbance, and vaginal/urinary symptoms\nGeneral Considerations\nA

In [21]:
from langchain_openai import ChatOpenAI
chatModel = ChatOpenAI(model="gpt-4.1-nano")

In [22]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [23]:
system_prompt = ("You are a medical assistant for question answering task"
                 "Answer the question based on the given context"
                 "Also suggest to visit doctor for severe cases you have to figure out that which case is severe"
                 "If you don't know the answer, just say that this is not to my knowledge nicely so you can consult to doctor for that"
                 "Answer in shortest possible manner but nicely"
                 "\n\n"
                 "{context}")

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),

    ])

In [24]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [27]:
response = rag_chain.invoke({"input": "What is labour pain?"})
print(response["answer"])

Labor pain is the discomfort experienced during labor, caused by ischemia of the uterus during contractions, dilation and effacement of the cervix, and distention of the vagina and perineum. If severe or persistent, consult a doctor.
